In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Tabou,CI,4.42,-7.35,75.65,79,73,7.94,broken clouds,0.0,0
1,1,Yellowknife,CA,62.46,-114.35,68.00,55,75,5.82,broken clouds,0.0,0
2,2,Kardla,EE,59.00,22.75,59.00,80,0,8.99,clear sky,0.0,0
3,3,Sistranda,NO,63.73,8.83,51.80,87,75,10.29,shower rain,0.0,0
4,4,Port Alfred,ZA,-33.59,26.89,57.67,43,0,13.94,clear sky,0.0,0


In [3]:
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain (inches)          float64
Snow (inches)            int64
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [7]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_question = str(input("Do you want it to be raining? (yes/no) "))
snow_question = str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [8]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp) & \
                                        (city_data_df["Rain (inches)"] == 0 if rain_question == "no" else city_data_df["Rain (inches)"] > 0)  & \
                                        (city_data_df["Snow (inches)"] == 0 if snow_question == "no" else city_data_df["Snow (inches)"] > 0) ]
                                    
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Tabou,CI,4.42,-7.35,75.65,79,73,7.94,broken clouds,0.0,0
12,12,Fitchburg,US,42.58,-71.80,88.00,45,40,11.41,scattered clouds,0.0,0
13,13,Cabadiangan,PH,9.75,122.47,77.07,86,100,2.86,overcast clouds,0.0,0
14,14,Krabi,TH,8.07,98.91,77.00,100,40,2.24,scattered clouds,0.0,0
31,31,Hithadhoo,MV,-0.60,73.08,84.78,71,30,7.09,scattered clouds,0.0,0
38,38,Hami,CN,42.80,93.45,77.13,35,52,6.91,broken clouds,0.0,0
42,42,Pacifica,US,37.61,-122.49,75.00,46,1,11.41,clear sky,0.0,0
47,47,La Joya,MX,18.67,-99.47,89.01,51,20,4.70,few clouds,0.0,0
48,48,Labuhan,ID,-6.88,112.21,76.53,85,98,7.56,overcast clouds,0.0,0
52,52,Raton,US,36.90,-104.44,86.00,35,1,9.17,clear sky,0.0,0


In [9]:
preferred_cities_df.count()

City_ID                168
City                   168
Country                168
Lat                    168
Lng                    168
Max Temp               168
Humidity               168
Cloudiness             168
Wind Speed             168
Current Description    168
Rain (inches)          168
Snow (inches)          168
dtype: int64

In [10]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tabou,CI,75.65,broken clouds,4.42,-7.35,
12,Fitchburg,US,88.00,scattered clouds,42.58,-71.80,
13,Cabadiangan,PH,77.07,overcast clouds,9.75,122.47,
14,Krabi,TH,77.00,scattered clouds,8.07,98.91,
31,Hithadhoo,MV,84.78,scattered clouds,-0.60,73.08,
38,Hami,CN,77.13,broken clouds,42.80,93.45,
42,Pacifica,US,75.00,clear sky,37.61,-122.49,
47,La Joya,MX,89.01,few clouds,18.67,-99.47,
48,Labuhan,ID,76.53,overcast clouds,-6.88,112.21,
52,Raton,US,86.00,clear sky,36.90,-104.44,


In [11]:
# Dependencies and Setup
import requests
import gmaps

# Import API key
from config import g_key

In [12]:
# Set the parameters to search for a hotel in Paris.
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key,
    "location": "48.8566, 2.3522"}
# Use base URL to search for hotels in Paris.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
# Make request and get the JSON data from the search.
hotels = requests.get(base_url, params=params).json()

In [13]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Tabou,CI,75.65,broken clouds,4.42,-7.35,hotêl doufoulougou
12,Fitchburg,US,88.00,scattered clouds,42.58,-71.80,Days Inn by Wyndham Leominster/Fitchburg Area
13,Cabadiangan,PH,77.07,overcast clouds,9.75,122.47,Kipot
14,Krabi,TH,77.00,scattered clouds,8.07,98.91,Tharapark View Hotel
31,Hithadhoo,MV,84.78,scattered clouds,-0.60,73.08,Scoop Guest House
38,Hami,CN,77.13,broken clouds,42.80,93.45,Laoyutang Resort
42,Pacifica,US,75.00,clear sky,37.61,-122.49,"Pacifica Lighthouse, Trademark Collection by W..."
47,La Joya,MX,89.01,few clouds,18.67,-99.47,HOTEL BUGAMBILIAS
48,Labuhan,ID,76.53,overcast clouds,-6.88,112.21,bador sepret
52,Raton,US,86.00,clear sky,36.90,-104.44,Holiday Inn Express & Suites Raton


In [15]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, 
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [17]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [18]:
# Create the output file (CSV).
output_data_file = "weather_data/WeatherPy_vacation.csv"